In [1]:

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import CuDNNGRU

from keras.layers import Dropout
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.callbacks import ModelCheckpoint

import numpy as np
import random
import sys
import io
import pandas as pd


Using TensorFlow backend.


At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

# load data

In [2]:
root_path = "../"
path_data_clean = root_path + "data/clean/"
dfNoticias = pd.read_pickle(path_data_clean + "/dfNoticiasCleanV2.p")
#dfNoticias.info()

# pre-procesado

In [3]:

raw_text = ""
count = 0
for index, row in dfNoticias.iterrows():    
    if index < 800: #max aporx 3500
        raw_text += row["Cuerpo"]
        
text = raw_text.lower()
print(len(text))
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


1277689
total chars: 90


In [4]:
# cut the text in semi-redundant sequences of maxlen characters

maxlen = 100 # 100 largo secuencia, 
step = 3 # 3pasos 


sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 425863
Vectorization...


# arquitectura red

# callbacks entrenamiento

In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0]:
    #for diversity in [0.5]:
        print('\n\n\n')
        print('----- diversidad:', diversity)
        print('\n\n')

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        #generated += sentence
        print('----- semilla: "' + sentence + '"')
        print('\n\n')
        sys.stdout.write(generated)

        for i in range(300):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

# updatable plot
# a minimal example (sort of)
import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output

class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="train_loss")
        plt.plot(self.x, self.val_losses, label="validation_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()    
        

def saveFullModelState(epoch, _):        
        model.save('weights/my_model.h5')
        
filepath="weights/weights.best.hdf5"
checkpointBest = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
        
filepath="weights/weights-improvement-{epoch:02d}-{loss:.4f}-test1.hdf5"
checkpointAll = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)







In [8]:

# build the model: a single LSTM

print('Build model...')

# model = Sequential()
# model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# model.add(Dropout(0.5))
# model.add(Dense(len(chars), activation='softmax'))


model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.5))
model.add(Dense(len(chars), activation='softmax'))



# model = Sequential()
# model.add(LSTM(128, input_shape=(maxlen, len(chars)) , return_sequences=True))
# model.add(Dropout(0.5))
# model.add(LSTM(64))
# model.add(Dropout(0.5))
# model.add(Dense(len(chars), activation='softmax'))


# model = Sequential()
# model.add(GRU(128, input_shape=(maxlen, len(chars)) , return_sequences=True))
# model.add(Dropout(0.5))
# model.add(GRU(64))
# model.add(Dropout(0.5))
# model.add(Dense(len(chars), activation='softmax'))



# model = Sequential()
# model.add(LSTM(128, input_shape=(maxlen, len(chars)) , return_sequences=True))
# model.add(Dropout(0.5))
# model.add(LSTM(128))
# model.add(Dropout(0.5))
# model.add(Dense(len(chars), activation='softmax'))


# model = Sequential()
# model.add(GRU(256, input_shape=(maxlen, len(chars)) , return_sequences=True))
# model.add(Dropout(0.5))
# model.add(GRU(256))
# model.add(Dropout(0.5))
# model.add(Dense(len(chars), activation='softmax'))



#model.compile(loss='categorical_crossentropy', optimizer='adam')




model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               355328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 90)                23130     
Total params: 378,458
Trainable params: 378,458
Non-trainable params: 0
_________________________________________________________________


In [7]:

model.load_weights("weights/weights.best.hdf5")
optimizer = RMSprop(lr=0.001) #0.01
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.fit(x, y,
          batch_size=1024,# 128
          epochs=60,
          validation_split=0.30,
          callbacks=[plot_losses,print_callback,checkpointAll,checkpointBest])

Train on 298104 samples, validate on 127759 samples
Epoch 1/60
116736/298104 [==========>...................] - ETA: 29s - loss: 1.4347

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [ ]:
import tensorflow as tf
print(tf.VERSION)